<a href="https://colab.research.google.com/github/akramIOT/IOT_BU/blob/master/ML_WIFI_Location_Tracking_Insights_RSSI_Scatter_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
print(tf.__version__)

!git clone --branch r1.13.0 --depth 1 https://github.com/tensorflow/models

2.2.0-rc4
fatal: destination path 'models' already exists and is not an empty directory.


In [0]:
## My environment is having Tensorflow 2.0 and Python 3.7 Installed, along with other frameworks.

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pandas as pd
import numpy as np
import gym
from sklearn.preprocessing import scale

dataset = pd.read_csv("/content/trainingData.csv",header = 0)

features = np.asarray(dataset.iloc[:,0:520])
features[features == 100] = -110
features = (features - features.mean()) / features.var()
print("features_mean:", features.mean())
print("features:", features.shape)

labels = np.asarray(dataset["BUILDINGID"].map(str) + dataset["FLOOR"].map(str))
print("Labels:", labels)
labels = np.asarray(pd.get_dummies(labels))

train_val_split = np.random.rand(len(features)) < 0.70
train_x = features[train_val_split]
print(train_x)
train_y = labels[train_val_split]
val_x = features[~train_val_split]
print(val_x)
val_y = labels[~train_val_split]

test_dataset = pd.read_csv("/content/validationData.csv",header = 0)

test_features = np.asarray(test_dataset.iloc[:,0:520])
test_features[test_features == 100] = -110
test_features = (test_features - test_features.mean()) / test_features.var()

test_labels = np.asarray(test_dataset["BUILDINGID"].map(str) + test_dataset["FLOOR"].map(str))
test_labels = np.asarray(pd.get_dummies(test_labels))

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

# Input and Hidden Layer details 

n_input = 520 
n_hidden_1 = 256 
n_hidden_2 = 128 
n_hidden_3 = 64 

n_classes = labels.shape[1]

learning_rate = 0.00001
training_epochs = 30
batch_size = 15

total_batches = train_x.shape[0] // batch_size

X = tf.placeholder(tf.float32, shape=[None,n_input])
Y = tf.placeholder(tf.float32,[None,n_classes])

# --------------------- Encoder Variables --------------- #
# Optimizing Bias and weights via  DNN is an important problem to solve

e_weights_h1 = weight_variable([n_input, n_hidden_1])
e_biases_h1 = bias_variable([n_hidden_1])

e_weights_h2 = weight_variable([n_hidden_1, n_hidden_2])
e_biases_h2 = bias_variable([n_hidden_2])

e_weights_h3 = weight_variable([n_hidden_2, n_hidden_3])
e_biases_h3 = bias_variable([n_hidden_3])

# --------------------- Decoder Variables --------------- #

d_weights_h1 = weight_variable([n_hidden_3, n_hidden_2])
d_biases_h1 = bias_variable([n_hidden_2])

d_weights_h2 = weight_variable([n_hidden_2, n_hidden_1])
d_biases_h2 = bias_variable([n_hidden_1])

d_weights_h3 = weight_variable([n_hidden_1, n_input])
d_biases_h3 = bias_variable([n_input])

# --------------------- DNN Variables ------------------ #

dnn_weights_h1 = weight_variable([n_hidden_3, n_hidden_2])
dnn_biases_h1 = bias_variable([n_hidden_2])

dnn_weights_h2 = weight_variable([n_hidden_2, n_hidden_2])
dnn_biases_h2 = bias_variable([n_hidden_2])

dnn_weights_out = weight_variable([n_hidden_2, n_classes])
dnn_biases_out = bias_variable([n_classes])


def encode(x):
    l1 = tf.nn.tanh(tf.add(tf.matmul(x,e_weights_h1),e_biases_h1))
    l2 = tf.nn.tanh(tf.add(tf.matmul(l1,e_weights_h2),e_biases_h2))
    l3 = tf.nn.tanh(tf.add(tf.matmul(l2,e_weights_h3),e_biases_h3))
    return l3
    
def decode(x):
    l1 = tf.nn.tanh(tf.add(tf.matmul(x,d_weights_h1),d_biases_h1))
    l2 = tf.nn.tanh(tf.add(tf.matmul(l1,d_weights_h2),d_biases_h2))
    l3 = tf.nn.tanh(tf.add(tf.matmul(l2,d_weights_h3),d_biases_h3))
    return l3

def dnn(x):
    l1 = tf.nn.tanh(tf.add(tf.matmul(x,dnn_weights_h1),dnn_biases_h1))
    l2 = tf.nn.tanh(tf.add(tf.matmul(l1,dnn_weights_h2),dnn_biases_h2))
    out = tf.nn.softmax(tf.add(tf.matmul(l2,dnn_weights_out),dnn_biases_out))
    return out

encoded = encode(X)
decoded = decode(encoded) 
y_ = dnn(encoded)

us_cost_function = tf.reduce_mean(tf.pow(X - decoded, 2))
s_cost_function = -tf.reduce_sum(Y * tf.log(y_))
us_optimizer = tf.train.AdamOptimizer(learning_rate).minimize(us_cost_function)
s_optimizer = tf.train.AdamOptimizer(learning_rate).minimize(s_cost_function)

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as session:
    tf.global_variables_initializer().run()
    
    # ------------ 1. Training Autoencoders - Unsupervised Learning ----------- #
    for epoch in range(training_epochs):
        epoch_costs = np.empty(0)
        for b in range(total_batches):
            offset = (b * batch_size) % (train_x.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :]
            _, c = session.run([us_optimizer, us_cost_function],feed_dict={X: batch_x})
            epoch_costs = np.append(epoch_costs,c)
        print("Epoch: ",epoch," Loss: ",np.mean(epoch_costs))
    print("Unsupervised pre-training finished...")
    
    
    # ---------------- 2. Training NN - Supervised Learning ------------------ #
    for epoch in range(training_epochs):
        epoch_costs = np.empty(0)
        for b in range(total_batches):
            offset = (b * batch_size) % (train_x.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :]
            batch_y = train_y[offset:(offset + batch_size), :]
            _, c = session.run([s_optimizer, s_cost_function],feed_dict={X: batch_x, Y : batch_y})
            epoch_costs = np.append(epoch_costs,c)
        print("Epoch: ",epoch," Loss: ",np.mean(epoch_costs)," Training Accuracy: ", \
            session.run(accuracy, feed_dict={X: train_x, Y: train_y}), \
            "Validation Accuracy:", session.run(accuracy, feed_dict={X: val_x, Y: val_y}))
            
    print("Supervised training finished...")
    

    print("\nTesting Accuracy:", session.run(accuracy, feed_dict={X: test_features, Y: test_labels}))


features_mean: -2.735186944520825e-17
features: (19937, 520)
Labels: ['12' '12' '12' ... '13' '13' '13']
[[-0.02819699 -0.02819699 -0.02819699 ... -0.02819699 -0.02819699
  -0.02819699]
 [-0.02819699 -0.02819699 -0.02819699 ... -0.02819699 -0.02819699
  -0.02819699]
 [-0.02819699 -0.02819699 -0.02819699 ... -0.02819699 -0.02819699
  -0.02819699]
 ...
 [-0.02819699 -0.02819699 -0.02819699 ... -0.02819699 -0.02819699
  -0.02819699]
 [-0.02819699 -0.02819699 -0.02819699 ... -0.02819699 -0.02819699
  -0.02819699]
 [-0.02819699 -0.02819699 -0.02819699 ... -0.02819699 -0.02819699
  -0.02819699]]
[[-0.02819699 -0.02819699 -0.02819699 ... -0.02819699 -0.02819699
  -0.02819699]
 [-0.02819699 -0.02819699 -0.02819699 ... -0.02819699 -0.02819699
  -0.02819699]
 [-0.02819699 -0.02819699 -0.02819699 ... -0.02819699 -0.02819699
  -0.02819699]
 ...
 [-0.02819699 -0.02819699 -0.02819699 ... -0.02819699 -0.02819699
  -0.02819699]
 [-0.02819699 -0.02819699 -0.02819699 ... -0.02819699 -0.02819699
  -0.028